In [ ]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
import networkx as nx
from scipy.spatial.distance import cdist
import sklearn
from sklearn.cluster import Birch, KMeans, MeanShift, DBSCAN, SpectralClustering
from sklearn.decomposition import TruncatedSVD
from normalized_cut import normalized_cut

from point_cloud_utils import get_pcd, transform_pcd
from aggregate_pointcloud import aggregate_pointcloud, aggregate_features
from reproject_merged_pointcloud import reproject_points_to_label, merge_associations, merge_features
from visualization_utils import generate_random_colors
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch

Here we define the dataset depending on kitti sequence!

In [ ]:
DATASET_PATH = os.path.join('/Users/laurenzheidrich/Downloads/','fused_dataset')
SEQUENCE_NUM = 7

config_filtered = KittiOdometryDatasetConfig(
    cache=True,
    dataset_path=DATASET_PATH,
    correct_scan_calibration=True,
    filters=FilterList(
        [
            KittiGTMovingObjectFilter(
                os.path.join(
                    DATASET_PATH,
                    "sequences",
                    "%.2d" % SEQUENCE_NUM,
                    "labels",
                )
            ),
            RangeFilter(3, 25),
        ]
    ),
)

dataset = KittiOdometryDataset(config_filtered, SEQUENCE_NUM)

Now we read in the point cloud and the left and right image of the stereo camera. If labels for those images are available they can be read in, too!

In [ ]:
ind_start = 0
ind_end = 200
voxel_size = 0.35

pcd, T_pcd, all_poses = aggregate_pointcloud(dataset, ind_start, ind_end, clip_to_imageframe=False, return_poses=True)
first_position = all_poses[0][:3,3]

num_points = np.asarray(pcd.points).shape[0]
print("num points: ", num_points)

In [ ]:
all_positions = []
for p in all_poses:
    all_positions.append(tuple(p[:3,3]))

sampled_indices_local = list(subsample_positions(all_positions, voxel_size=1))
sampled_indices_global = list(subsample_positions(all_positions, voxel_size=1) + ind_start)

poses = np.array(all_poses)[sampled_indices_local]
positions = np.array(all_positions)[sampled_indices_local]

In [ ]:
chunk_size = np.array([10, 14, 10]) #meters
overlap = 3 #meters

pcd_chunks, center_positions, center_ids = chunks_from_pointcloud(pcd, T_pcd, positions, poses, first_position, 
                                                                  sampled_indices_global, chunk_size, overlap)

pcd_chunks_downsampled = []

voxel_size = 0.35
for chunk in pcd_chunks:
    pcd_downsample = chunk.voxel_down_sample(voxel_size=voxel_size)
    pcd_chunks_downsampled.append(pcd_downsample)
    print("Downsampled from", np.asarray(chunk.points).shape, "to", np.asarray(pcd_downsample.points).shape, "points")

In [ ]:
patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)

In [ ]:
for sequence in range(len(center_ids)):

    print("Start of sequence ", sequence)

    first_id = patchwise_indices[sequence][0]
    center_id = center_ids[sequence]
    center_position = center_positions[sequence]

    pcd_chunk = pcd_chunks[sequence]
    downsampled_chunk = pcd_chunks_downsampled[sequence]

    points = np.asarray(downsampled_chunk.points)
    num_points = np.asarray(downsampled_chunk.points).shape[0]    

    print(num_points, "points in downsampled chunk")

    tarl_features = tarl_features_per_patch(dataset, downsampled_chunk, center_id, T_pcd, center_position,
                                            sampled_indices_global, chunk_size, voxel_size)

    cams = ["cam2", "cam3"]
    sam_feature_reprojections, dinov2_feature_reprojections = image_based_features_per_patch(dataset, downsampled_chunk, T_pcd, sampled_indices_global, first_id, cams, cam_id=0)

    sam_features = merge_associations(sam_feature_reprojections, len(downsampled_chunk.points))
    dinov2_features = merge_features(dinov2_feature_reprojections, len(downsampled_chunk.points))

    spatial_distance = cdist(points, points)
    dinov2_distance = cdist(dinov2_features, dinov2_features)
    tarl_distance = cdist(tarl_features, tarl_features)

    proximity_threshold = 2 # meters that points can be apart from each other and still be considered neighbors
    alpha = 2.3 # weight of the spatial proximity term  2.3
    beta = 0.9 # weight of the label similarity term 1.1
    gamma = 0.5 #0.3 # weight of the dinov2 feature similarity term 1.5
    theta = 1.4 # weight of the tarl feature similarity term

    sam_edge_weights, mask = sam_label_distance(sam_features, spatial_distance, proximity_threshold, num_points, beta)
    spatial_edge_weights = np.exp(-alpha * (mask * spatial_distance))
    dinov2_edge_weights = np.exp(-gamma * (mask * dinov2_distance))
    tarl_edge_weights = np.exp(-theta * (mask * tarl_distance))

    A = spatial_edge_weights * sam_edge_weights * dinov2_edge_weights * tarl_edge_weights
    print("Adjacency Matrix built")

    # Remove isolated points
    isolated_mask = ~np.all(A == 0, axis=1)
    A = A[isolated_mask][:, isolated_mask]
    downsampled_chunk = downsampled_chunk.select_by_index(np.where(isolated_mask == True)[0])
    print(num_points - np.asarray(downsampled_chunk.points).shape[0], "isolated points removed")
    num_points = np.asarray(downsampled_chunk.points).shape[0]

    print("Start of normalized Cuts")
    grouped_labels = normalized_cut(A, np.arange(num_points), T = 0.00001)

    print("There are", len(grouped_labels), "cut regions")

    random_colors = generate_random_colors(600)

    pcd_color = np.zeros((num_points, 3))

    for i, s in enumerate(grouped_labels):
        for j in s:
            pcd_color[j] = np.array(random_colors[i]) / 255

    downsampled_chunk.colors = o3d.utility.Vector3dVector(pcd_color)

    pcd_chunk.paint_uniform_color([0, 0, 0])
    pcd_tree = o3d.geometry.KDTreeFlann(downsampled_chunk)

    i=0
    for point in np.asarray(pcd_chunk.points):
        [_, idx, _] = pcd_tree.search_knn_vector_3d(point, 1)
        np.asarray(pcd_chunk.colors)[i,:] = np.asarray(downsampled_chunk.colors)[idx[0], :]
        i+=1

    index_file = str(center_id).zfill(6) + '.pcd'
    file = os.path.join("test_data", index_file)
    o3d.io.write_point_cloud(file, pcd_chunk, write_ascii=False, compressed=False, print_progress=False)
    
    print("Pointcloud written to file")
    print("End of sequence ", sequence)